In [ ]:
from iminuit import cost
from iminuit import Minuit
from numba_stats import t, bernstein, truncexpon
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
def sig1(xe, mu, sigma, nuinv):
    nu = 1 / nuinv
    return t.cdf(xe, nu, mu, sigma)

def bkg1(xe, b1, b2, b3):
    return bernstein.integral(xe, (b1, b2, b3), 0, 1)

def model(xe, s, mu, sigma, nuinv, b1, b2, b3):
    sn = s * sig1(xe, mu, sigma, nuinv)
    bn = bkg1(xe, b1, b2, b3)
    return sn + bn

rng = np.random.default_rng(1)

truth = 1000., 0.5, 0.1, 0.1, 1000., 3000., 2000.

xe = np.linspace(0, 1, 100)
sm = truth[0] * np.diff(sig1(xe, *truth[1:4]))
bm = np.diff(bkg1(xe, *truth[4:]))
n = rng.poisson(np.diff(model(xe, *truth)))

c = cost.ExtendedBinnedNLL(n, xe, model)

m = Minuit(c, *truth)
m.limits["s", "sigma", "nuinv", "b1", "b2", "b3"] = (0, None)
m.limits["mu"] = (0, 1)
m.limits["nuinv"] = (0, 5)

m.interactive()

In [ ]:
cx = 0.5 * (xe[1:] + xe[:-1])
c.mask = np.abs(cx - 0.5) > 0.3
m.interactive()

In [ ]:
def sig2(xe, mu, sigma, nuinv):
    nu = 1 / nuinv
    a, b = t.cdf((0, 1), nu, mu, sigma)
    return (t.cdf(xe, nu, mu, sigma) - a) / (b - a)

def bkg2(xe, slope):
    return truncexpon.cdf(xe, 0, 1, 0, slope)

def model(xe, f, mu, sigma, nuinv, slope):
    sn = f * sig2(xe, mu, sigma, nuinv)
    bn = (1 - f) * bkg2(xe, slope)
    return sn + bn

rng = np.random.default_rng(1)

truth = 0.5, 0.5, 0.1, 0.1, 1

xe = np.linspace(0, 1, 100)
sm = truth[0] * np.diff(sig2(xe, *truth[1:4]))
bm = (1 - truth[0]) * np.diff(bkg2(xe, *truth[4:]))
n = rng.poisson(1000 * np.diff(model(xe, *truth)))

c = cost.BinnedNLL(n, xe, model)

m = Minuit(c, *truth)
m.limits["sigma", "slope"] = (1e-3, None)
m.limits["mu", "f"] = (0, 1)
m.limits["nuinv"] = (0, 0.2)

m.interactive()

In [ ]:
cx = 0.5 * (xe[1:] + xe[:-1])
c.mask = np.abs(cx - 0.5) > 0.3
m.interactive()

In [ ]:
c = cost.BarlowBeestonLite(n, xe, (10000 * bm, 1000 * sm))
m = Minuit(c, 500, 500)
m.interactive()

In [ ]:
def model(x, a, b):
    return a + b * x

truth = (1., 2.)
x = np.linspace(0, 1)
ym = model(x, *truth)
ye = 0.1
y = rng.normal(ym, ye)

c = cost.LeastSquares(x, y, ye, model)

m = Minuit(c, *truth)
m.interactive()

In [ ]:
c.mask = (x > 0.6) | (x < 0.2)
m = Minuit(c, *truth)
m.interactive()